In [27]:
import pandas as pd
import subprocess


In [28]:
def get_elements_by_pattern(elements: list[str], beginning, end) -> list[str]:
    orig_query_line = list(map(
        lambda s: s[s.index(beginning) + len(beginning) : s.index(end, s.index(beginning))],
        filter(
            lambda s: beginning in s and end in s and s.index(beginning) < s.index(end, s.index(beginning)),
            elements
        )
    ))
    
    if len(orig_query_line) < 1:
        return elements[-1].split('[ANALYSIS] ast: ', 1)[1]
    # make sure exactly one output of this type can be found
    return orig_query_line



def save_analysis(output):
    analysis = pd.DataFrame()
    results = list(filter(lambda x: "[ANALYSIS]" in x, output.split("\n")))
    analysis["query number"] = get_elements_by_pattern(results, '[ANALYSIS] QUERY PATH: "/workspaces/reducer/queries/query', '/original_test.sql"[END ANALYSIS]')

    analysis["original"] = get_elements_by_pattern(results, '[ANALYSIS] ORIGINAL QUERY: "', '"[END ANALYSIS]')
    analysis["delta debugged"] = get_elements_by_pattern(results, '[ANALYSIS] AFTER DELTA DEBUGGING: ', '[END ANALYSIS]')
    #analysis["reduced"] = []

    analysis["#statements original"] = analysis["original"].apply(lambda x: x.count(";"))
    analysis["#statements delta debugged"] = analysis["delta debugged"].apply(lambda x: x.count(";"))
    analysis["#tokens original"] = analysis["original"].apply(lambda s: len(s.split(" ")))

    analysis["DIFF orig-delta"] = analysis["#statements original"] - analysis["#statements delta debugged"]
    return analysis
    

In [29]:
def build_and_prepare():
    cmd = """
    export CARGO_TARGET_DIR=/target
    export RUST_LOG=info
    cargo build
    """
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, executable="/bin/bash")
    print(result)

def run_multiple_queries(start=1, stop=21, step=1):
    outputs = []
    for i in range(start, stop, step):
        outputs.append(run_single_query(i))
    return outputs

def run_single_query(query_number):
    cmd = f"""
    cd ../..
    export CARGO_TARGET_DIR=/target
    export RUST_LOG=warn
    cargo build
    cargo run --package reducer --bin reducer -- --query queries/query{query_number}/original_test.sql --test src/resources/native.sh
 
    """
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    output = result.stdout + result.stderr  # Capture both stdout and stderr if needed
    return output

In [30]:
build_and_prepare()
output = run_multiple_queries()

CompletedProcess(args='\n    export CARGO_TARGET_DIR=/target\n    export RUST_LOG=info\n    cargo build\n    ', returncode=0, stdout='', stderr='\x1b\x1b   Compiling\x1b reducer v0.1.0 (/workspaces/reducer)\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `std::sync::OnceLock`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/driver.rs:8:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b8\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse std::sync::OnceLock;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^^^^^^^^^^^\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b  \x1b\x1b\x1b\x1b= \x1b\x1b\x1bnote\x1b\x1b: `#[warn(unused_imports)]` on by default\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: `log::info`\x1b\n\x1b \x1b\x1b\x1b\x1b--> \x1b\x1bsrc/parser.rs:1:5\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\n\x1b\x1b\x1b1\x1b\x1b \x1b\x1b\x1b\x1b|\x1b\x1b \x1b\x1buse log::info;\x1b\n\x1b  \x1b\x1b\x1b\x1b|\x1b\x1b     \x1b\x1b\x1b\x1b^^^^^^^^^\x1b\n\n\x1b\x1b\x1bwarning\x1b\x1b\x1b: unused import: 

In [31]:
results = pd.DataFrame()
if isinstance(output, str):
    results = save_analysis(output)
else:
    for out in output:
        results = pd.concat([results, save_analysis(out)], ignore_index=True)
results

query number  \
0             1   
1             2   
2             3   
3             4   
4             5   
5             6   
6             7   
7             8   
8             9   
9            10   
10           11   
11           12   
12           13   
13           14   
14           15   
15           16   
16           17   
17           18   
18           19   
19           20   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [32]:
pd.set_option('display.max_colwidth', None)

# (Optionally) expand the total width so columns don’t wrap
pd.set_option('display.width', 200)
results[results['#statements delta debugged'] == 0]["delta debugged"]

Series([], Name: delta debugged, dtype: object)